### queries.jsonl -> queries-edited.jsonl 

queries.jsonl : 형법.xlsx -> queries.jsonl

queries-edited.jsonl은 형법.jsonl 에서 아래 규칙을 적용하여,  
사람이 직접 검토 수정한 데이터. 코드로는 미구현됨   

1. 형법 제1편 제외, 기타 조항 제외.  

형에 대한 조항보다 무엇이 범법행위에 해당하는 지 위주로 진행  
id 472 부터 미수범, 예비음모 처리X (아예 제외)  
id 512 부터 병과 처리 X  
id 545 부터 상습범 처리 X  
형의 감경 처리 X  
세계주의 처리 X  
친족간의 범행 처리 X  

2. Query expansion  

-의 예에 의한다, 전항의 예에 따른다 등 참조표현 대체 및 정보추가.  

In [1]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("json", data_files="../../dataset/preprocessing/gpt_input/queries-edited.jsonl")['train'] # query expansion 된 데이터

df = pd.DataFrame(dataset)
df

c:\Users\fehs0\miniconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,title,content
0,219,형법 제87조,제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문...
1,220,형법 제87조 제1호,우두머리는 사형 무기징역 또는 무기금고에 처한다
2,221,형법 제87조 제2호,모의에 참여하거나 지휘하거나 그 밖의 중요한 임무에 종사한 자는 사형 무기 또는 5...
3,222,형법 제87조 제3호,부화수행(附和隨行)하거나 단순히 폭동에만 관여한 자는 5년 이하의 징역이나 금고에 처한다
4,223,형법 제88조,제88조(내란목적의 살인) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나...
...,...,...,...
358,755,형법 제368조 제2항,재물손괴죄 또는 중손괴의 죄를 범하여 사람을 상해에 이르게 한 때에는 1년 이상의 ...
359,757,형법 제369조 제1항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 재물손괴죄를 범한 때에는 ...
360,758,형법 제369조 제2항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 중손괴죄를 범한 때에는 1...
361,759,형법 제370조,제370조(경계침범) 경계표를 손괴 이동 또는 제거하거나 기타 방법으로 토지의 경계...


## GPT 문장 생성으로 query-pos 튜닝 데이터셋 구축

### Asynchronous API 요청

In [ ]:
from openai import AsyncOpenAI
from pydantic import BaseModel
import pandas as pd
import json
import os
from tqdm.asyncio import tqdm_asyncio
import asyncio

class Triplet(BaseModel):
    id : int
    title : str
    query: str
    pos: list[str]

async def process_row_async(row, instructions, client):
    id = row['id']
    title = row['title']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    try:
        response = await client.responses.parse(
            model='gpt-5-mini',
            instructions=instructions,
            input=[{"role": "user", "content": prompt}],
            text_format=Triplet,
        )
        return dict(response.output_parsed)
    except Exception as e:
        print(f"Error in row {title} : {e}")
        return {'id': id , 'title': title, 'query': query, 'pos': []}
    
async def generate_sentences_async(instrctions, client, queries : pd.DataFrame, dir_path: str,concurrency: int = 6):
    sem = asyncio.Semaphore(concurrency) # 병렬 처리 수 제한

    async def sem_task(row):
        async with sem:
            return await process_row_async(row, instrctions, client)
        
    tasks = [sem_task(row) for _, row in queries.iterrows()]
    results = []

    for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Processing"):
        result = await coro
        if result:
            results.append(result)

    os.makedirs(dir_path, exist_ok=True)
    fname = "incidents.jsonl"
    path = os.path.join(dir_path, fname)

    # 비동기 처리 결과값들 원래 쿼리 데이터 순서대로 정렬
    sorted_results = sorted(results, key=lambda x : x['id'])

    with open(path, 'w', encoding='utf-8') as f:
        for res in sorted_results:
            json.dump(res, f, ensure_ascii=False)
            f.write('\n')

문장 생성

In [6]:
from dotenv import load_dotenv
import nest_asyncio

load_dotenv()

# k = 5
k = 20 # 10개는 파인튜닝, 나머지 10개는 validation set으로 사용

client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

with open("gpt_prompt.txt", "r", encoding="utf-8") as f:
    instructions = f.read().replace("#K#", str(k))

nest_asyncio.apply() # jupyter notebook 자체적인 running event loop 가 존재하므로 실행 중 루프 (주피터 노트북) 내 중첩된 루프를 허용

asyncio.run(generate_sentences_async(instructions, client, df, dir_path="../../dataset/preprocessing/gpt_output")) # incidents.jsonl

Processing: 100%|██████████| 363/363 [27:51<00:00,  4.61s/it]


### 기존 쿼리 jsonl 파일 데이터 포맷 변경 (선택사항)  

조항 구분 띄어쓰기 제거  

In [ ]:
from datasets import load_dataset
import os
import re


def trans_format(fname: str, law_kind: str):
    ds = load_dataset("json", data_files=fname)["train"]
    new_ds = ds.map(
        lambda x: {
            "id": x["id"],
            "title": f"{law_kind} {re.sub(r'(?<!^)(?=제)', ' ', x['query']['title'])}",  # 띄어쓰기 제거
            "content": x["query"]["content"],
        },
    )
    new_ds = new_ds.remove_columns(["query"])
    new_fname = os.path.splitext(fname)[0] + "_2" + ".jsonl"
    new_ds.to_json(new_fname, force_ascii=False)


trans_format("../../dataset/preprocessing/queries-edited.jsonl", "형법")

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]